In [83]:
# Importing necessary libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf

In [84]:
# Loading the train dataset
data = pd.read_csv("./train.csv")

In [85]:
# Data preview
data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [86]:
# Data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [87]:
# Null value check
data.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [88]:
# Inspect label value counts
data["price_range"].value_counts()

1    500
2    500
3    500
0    500
Name: price_range, dtype: int64

In [89]:
# Split features and labels
X = data.drop("price_range", axis=1)
y = data["price_range"]

In [90]:
# Spliting the dataset into the training set and validation set
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [91]:
# Feature Scaling

mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_valid = mms.fit_transform(X_valid)

In [92]:
# K-Fold Initialization
kfold = KFold(n_splits=10, shuffle=True)

acc_per_fold = []
loss_per_fold = []

In [93]:
# K-Fold on action

n_fold = 1
inputs = np.concatenate((X_train, X_valid), axis=0)
targets = np.concatenate((y_train, y_valid), axis=0)

for train, valid in kfold.split(inputs, targets):
    ann = tf.keras.models.Sequential()
    # Input Layer
    ann.add(tf.keras.Input(shape=(20,)))
    # Hidden Layer
    ann.add(tf.keras.layers.Dense(units=32, activation="relu")) 
    # Output Layer
    ann.add(tf.keras.layers.Dense(units =4, activation="softmax"))

    ann.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
    
    ann.fit(inputs[train], targets[train], batch_size=32, epochs=100)

    # Generalization metrics
    scores = ann.evaluate(inputs[valid], targets[valid], verbose=0)

    print(f'Score for fold {n_fold}: {ann.metrics_names[0]} of {scores[0]}; {ann.metrics_names[1]} of {scores[1]*100}%')
    
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    n_fold += 1

Epoch 1/100
57/57 [==============================] - 0s 1ms/step - loss: 1.4247 - accuracy: 0.2500
Epoch 2/100
57/57 [==============================] - 0s 2ms/step - loss: 1.3621 - accuracy: 0.3167
Epoch 3/100
57/57 [==============================] - 0s 2ms/step - loss: 1.3014 - accuracy: 0.4022
Epoch 4/100
57/57 [==============================] - 0s 1ms/step - loss: 1.2291 - accuracy: 0.4978
Epoch 5/100
57/57 [==============================] - 0s 1ms/step - loss: 1.1599 - accuracy: 0.5428
Epoch 6/100
57/57 [==============================] - 0s 1ms/step - loss: 1.0884 - accuracy: 0.5911
Epoch 7/100
57/57 [==============================] - 0s 1ms/step - loss: 1.0256 - accuracy: 0.6500
Epoch 8/100
57/57 [==============================] - 0s 2ms/step - loss: 0.9659 - accuracy: 0.6494
Epoch 9/100
57/57 [==============================] - 0s 1ms/step - loss: 0.9124 - accuracy: 0.6833
Epoch 10/100
57/57 [==============================] - 0s 1ms/step - loss: 0.8668 - accuracy: 0.7211
Epoch 11/

In [94]:
# Validation Results

print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.1928069293498993 - Accuracy: 93.99999976158142%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.20451314747333527 - Accuracy: 92.00000166893005%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.17230267822742462 - Accuracy: 95.99999785423279%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.17012923955917358 - Accuracy: 96.49999737739563%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.20837527513504028 - Accuracy: 91.50000214576721%
------------------------------------------------------------------------
> Fold 6 - Loss: 0.20783483982086182 - Accuracy: 93.00000071525574%
-------------------------------------------------------------------

In [95]:
# Build model

ann = tf.keras.models.Sequential()
# Input Layer
ann.add(tf.keras.Input(shape=(20,)))
# Hidden Layer
ann.add(tf.keras.layers.Dense(units=32, activation="relu")) 
# Output Layer
ann.add(tf.keras.layers.Dense(units =4, activation="softmax"))

ann.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
    
ann.fit(inputs[train], targets[train], batch_size=32, epochs=100)

# Save Model
ann.save("./ann")

Epoch 1/100
57/57 [==============================] - 0s 2ms/step - loss: 1.3928 - accuracy: 0.3094
Epoch 2/100
57/57 [==============================] - 0s 1ms/step - loss: 1.3219 - accuracy: 0.3983
Epoch 3/100
57/57 [==============================] - 0s 1ms/step - loss: 1.2630 - accuracy: 0.4533
Epoch 4/100
57/57 [==============================] - 0s 1ms/step - loss: 1.1913 - accuracy: 0.5183
Epoch 5/100
57/57 [==============================] - 0s 1ms/step - loss: 1.1142 - accuracy: 0.5539
Epoch 6/100
57/57 [==============================] - 0s 1ms/step - loss: 1.0357 - accuracy: 0.5994
Epoch 7/100
57/57 [==============================] - 0s 1ms/step - loss: 0.9623 - accuracy: 0.6567
Epoch 8/100
57/57 [==============================] - 0s 1ms/step - loss: 0.8971 - accuracy: 0.6561
Epoch 9/100
57/57 [==============================] - 0s 2ms/step - loss: 0.8422 - accuracy: 0.7039
Epoch 10/100
57/57 [==============================] - 0s 1ms/step - loss: 0.7933 - accuracy: 0.7378
Epoch 11/

In [96]:
# Load model
model = tf.keras.models.load_model("./ann")

In [97]:
# Load test data
test = pd.read_csv("./test.csv")
test = test.drop("id", axis=1)

In [98]:
# Make predictions
test.shape
predictions = model.predict(test)
predictions

32/32 [==============================] - 0s 1ms/step


array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)